In [1]:
import numpy as np
from scipy.linalg import norm
import re

In [2]:
H_matrix=1/np.sqrt(2)*np.array([[1, 1],
                                [1,-1]])

CNOT_matrix=np.array([[1,0,0,0],
                      [0,1,0,0],
                      [0,0,0,1],
                      [0,0,1,0]])

CNOT_tensor=np.reshape(CNOT_matrix, (2,2,2,2))

In [14]:
class Reg: 
    def __init__(self,n):
        self.n=n
        self.psi=np.zeros((2,)*n) 
        self.psi[(0,)*n]=1
        
def H(i,reg): 
    reg.psi=np.tensordot(H_matrix,reg.psi,(1,i)) 
    reg.psi=np.moveaxis(reg.psi,0,i)

def CNOT(control, target, reg):
    reg.psi=np.tensordot(CNOT_tensor, reg.psi, ((2,3),(control, target))) 
    reg.psi=np.moveaxis(reg.psi,(0,1),(control,target))   

def depolarize_channel(i,reg, nu): 
    projectors=[ (1-nu)*np.array([[1,0],[0,1]]), nu * np.array([[0.5,0.5],[0.5,0.5]]) ] 
    
    def project(i,j,reg): 
        projected=np.tensordot(projectors[j],reg.psi,(1,i))
        return np.moveaxis(projected,0,i)
    
    projected=project(i,0,reg) 
    norm_projected=norm(projected.flatten()) 
    if np.random.random()<norm_projected**2: 
        reg.psi=projected/norm_projected
        return 0
    else:
        projected=project(i,1,reg)
        reg.psi=projected/norm(projected)
        return 1
    

In [15]:
def extract_gate_instructions(file_path):
    # Read the text file
    with open(file_path, 'r') as file:
        data = file.read()

    # Extract number of qubits
    num_qubits = int(re.search(r'qubits (\d+)', data).group(1))

    # Extract gate instructions
    gate_instructions = []
    matches = re.findall(r'(\w+)\s+(\d+),?\s?(\d*)', data)
    for match in matches:
        gate = match[0]
        qubits = [int(q) for q in match[1:] if q]
        gate_instructions.append((gate, qubits))

    return num_qubits, gate_instructions[1:]

In [28]:
def execute_gate_instructions(file_path,nu):

    num_qubits, gate_instructions = extract_gate_instructions(file_path)

 
    reg = Reg(num_qubits)

    for gate, qubits in gate_instructions:
        if gate == 'h':
            H(qubits[0], reg)
            for i in range(num_qubits):
                c = depolarize_channel(i,reg, nu)
                print(c)
        elif gate == 'cx':
            control, target = qubits
            CNOT(control, target, reg)
            for i in range(num_qubits):
                c = depolarize_channel(i,reg, nu)
                print(c)

    return reg.psi.flatten()

In [29]:
execute_gate_instructions('input.txt', nu = 0.1)

0
1
1
0
0
0
0
0
0


array([0.35355339, 0.35355339, 0.35355339, 0.35355339, 0.35355339,
       0.35355339, 0.35355339, 0.35355339])